In [1]:
import pandas as pd
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re

In [2]:
train_df = pd.read_csv("./data/ratings_train.txt", sep = "\t")
test_df = pd.read_csv("./data/ratings_test.txt", sep = "\t")

In [3]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [5]:
# 결측치 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [6]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [7]:
train_df = train_df.dropna(subset = ["document"])
test_df = test_df.dropna(subset = ["document"])

In [8]:
train_df.shape, test_df.shape

((149995, 3), (49997, 3))

In [9]:
# 종속변수 확인
np.unique(train_df["label"], return_counts = True)

(array([0, 1], dtype=int64), array([75170, 74825], dtype=int64))

In [10]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


# 데이터 전처리

In [11]:
# 한글 이외의 문자들 제거
train_df["document"] = train_df["document"].map(lambda x: re.sub("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣 ]", "", x))
test_df["document"] = test_df["document"].map(lambda x: re.sub("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣 ]", "", x))

In [12]:
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [13]:
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


In [14]:
train_df = train_df[train_df["document"].map(lambda x: len(x.strip()) >= 1)]
test_df = test_df[test_df["document"].map(lambda x: len(x.strip()) >= 1)]

In [15]:
train_df.shape, test_df.shape

((148740, 3), (49575, 3))

In [16]:
# 중복 데이터 확인
train_df[train_df["document"].duplicated(keep = False)].sort_values("document")

,id,document,label
46599,9682597,그리고 내 감정을 불러 일으켰다,1
43436,9582856,그리고 내 감정을 불러 일으켰다,1
123713,9582855,그리고 내 감정을 불러 일으켰다,1
93364,171409,가입 추천바람,1
138373,171407,가입 추천바람,1
...,...,...,...
57831,3906478,흥미진진,1
8446,5158304,힐러리 더프의 매력에 빠지다,1
72688,5153363,힐러리 더프의 매력에 빠지다,1
26889,7971814,힘들다,0


In [17]:
# 중복제거
train_df = train_df.drop_duplicates(subset = ["document"], keep ="first")
test_df = test_df.drop_duplicates(subset = ["document"], keep ="first")

In [18]:
train_df.shape, test_df.shape

((143660, 3), (48403, 3))

# 토큰화

In [52]:
okt = Okt()

In [20]:
okt.morphs("아 더빙 진짜 짜증나네요 목소리", stem = False)

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [21]:
okt.morphs("아 더빙 진짜 짜증나네요 목소리", stem = True)

['아', '더빙', '진짜', '짜증나다', '목소리']

In [22]:
okt.nouns("아 더빙 진짜 짜증나네요 목소리")

['더빙', '진짜', '목소리']

In [23]:
okt.pos("아 더빙 진짜 짜증나네요 목소리")

[('아', 'Exclamation'),
 ('더빙', 'Noun'),
 ('진짜', 'Noun'),
 ('짜증나네요', 'Adjective'),
 ('목소리', 'Noun')]

In [24]:
%%time
train_df["token"] = train_df["document"].map(lambda x: okt.morphs(x, stem = True))
test_df["token"] = test_df["document"].map(lambda x: okt.morphs(x, stem = True))

CPU times: total: 22min 43s
Wall time: 21min 58s


In [27]:
train_df.head(20)

,id,document,label,token
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"[아, 더빙, 진짜, 짜증나다, 목소리]"
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1,"[흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 조차, 가볍다, 않다]"
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[너, 무재, 밓었, 다그, 래서, 보다, 추천, 한, 다]"
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0,"[교도소, 이야기, 구먼, 솔직하다, 재미, 는, 없다, 평점, 조정]"
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1,"[사이, 몬페, 그, 의, 익살스럽다, 연기, 가, 돋보이다, 영화, 스파이더맨, ..."
5,5403919,막 걸음마 뗀 세부터 초등학교 학년생인 살용영화ㅋㅋㅋ별반개도 아까움,0,"[막, 걸음, 마, 떼다, 세, 부터, 초등학교, 학년, 생인, 살다, 영화, ㅋㅋ..."
6,7797314,원작의 긴장감을 제대로 살려내지못했다,0,"[원작, 의, 긴장감, 을, 제대로, 살리다, 하다]"
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫...,0,"[별, 반개, 도, 아깝다, 욕, 나오다, 이응경, 길용우, 연, 기, 생활, 이,..."
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1,"[액션, 이, 없다, 재미, 있다, 몇, 안되다, 영화]"
9,5912145,왜케 평점이 낮은건데 꽤 볼만한데 헐리우드식 화려함에만 너무 길들여져 있나,1,"[왜케, 평점, 이, 낮다, 꽤, 볼, 만, 한, 데, 헐리우드, 식, 화려하다, ..."


# 맛집 코드

In [3]:
train_df.to_csv("nsmc_ratings_train_pre.csv", index = False)
test_df.to_csv("nsmc_ratings_test_pre.csv", index = False)

In [4]:
train_df = pd.read_csv("./nsmc_ratings_train_pre.csv")
test_df = pd.read_csv("./nsmc_ratings_test_pre.csv")

In [8]:
type(train_df.loc[0, "token"])

list

In [6]:
# 위 타입이 문자열이라면 리스트로 바꾸는 법
train_df["token"] = train_df["token"].map(lambda x: eval(x))
test_df["token"] = test_df["token"].map(lambda x: eval(x))

In [7]:
train_df.shape, test_df.shape

((143660, 4), (48403, 4))

train_df["token"] = train_df["token"].map(lambda x: [i for i in x if len(i) > 1])
test_df["token"] = test_df["token"].map(lambda x: [i for i in x if len(i) > 1])

In [9]:
train_df.shape, test_df.shape

((143660, 4), (48403, 4))

In [10]:
train_df.head()

,id,document,label,token
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"[아, 더빙, 진짜, 짜증나다, 목소리]"
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1,"[흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 조차, 가볍다, 않다]"
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[너, 무재, 밓었, 다그, 래서, 보다, 추천, 한, 다]"
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0,"[교도소, 이야기, 구먼, 솔직하다, 재미, 는, 없다, 평점, 조정]"
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1,"[사이, 몬페, 그, 의, 익살스럽다, 연기, 가, 돋보이다, 영화, 스파이더맨, ..."


In [11]:
train_df = train_df[train_df["token"]. map(lambda x: len(x) > 0)]
test_df = test_df[test_df["token"]. map(lambda x: len(x) > 0)]

In [12]:
train_df.shape, test_df.shape

((143660, 4), (48403, 4))

# 정수인코딩

In [13]:
tokenizer = Tokenizer()

In [14]:
# 단어 집합 생성
# 등장 빈도 수가 높은 순서대로 정수값 부여
tokenizer.fit_on_texts(train_df["token"])

In [15]:
# 단어 집합
tokenizer.word_index

{'이': 1,
 '영화': 2,
 '보다': 3,
 '하다': 4,
 '의': 5,
 '에': 6,
 '가': 7,
 '을': 8,
 '도': 9,
 '들': 10,
 '는': 11,
 '를': 12,
 '은': 13,
 '없다': 14,
 '이다': 15,
 '있다': 16,
 '좋다': 17,
 '너무': 18,
 '다': 19,
 '정말': 20,
 '한': 21,
 '되다': 22,
 '적': 23,
 '만': 24,
 '재밌다': 25,
 '같다': 26,
 '진짜': 27,
 '으로': 28,
 '로': 29,
 '아니다': 30,
 '않다': 31,
 '점': 32,
 '에서': 33,
 '만들다': 34,
 '과': 35,
 '나오다': 36,
 '연기': 37,
 '것': 38,
 '평점': 39,
 '내': 40,
 '최고': 41,
 '그': 42,
 '나': 43,
 '안': 44,
 '인': 45,
 '스토리': 46,
 '생각': 47,
 '못': 48,
 '왜': 49,
 '드라마': 50,
 '게': 51,
 '사람': 52,
 '감동': 53,
 '보고': 54,
 '이렇다': 55,
 '고': 56,
 '말': 57,
 '아깝다': 58,
 '더': 59,
 '배우': 60,
 '때': 61,
 'ㅋㅋ': 62,
 '와': 63,
 '아': 64,
 '감독': 65,
 '거': 66,
 '그냥': 67,
 '요': 68,
 '재미있다': 69,
 '재미': 70,
 '시간': 71,
 '내용': 72,
 '까지': 73,
 '뭐': 74,
 '중': 75,
 '주다': 76,
 '좀': 77,
 '자다': 78,
 '하고': 79,
 '지루하다': 80,
 '수': 81,
 '재미없다': 82,
 '네': 83,
 '쓰레기': 84,
 '모르다': 85,
 '가다': 86,
 '들다': 87,
 '그렇다': 88,
 '싶다': 89,
 '지': 90,
 '작품': 91,
 '사랑': 92,
 '알다': 93,
 '하나': 94

In [16]:
# 단어 등장 수
tokenizer.word_counts

OrderedDict([('아', 4121),
             ('더빙', 572),
             ('진짜', 8288),
             ('짜증나다', 1002),
             ('목소리', 374),
             ('흠', 246),
             ('포스터', 572),
             ('보고', 4653),
             ('초딩', 422),
             ('영화', 50172),
             ('줄', 1240),
             ('오버', 142),
             ('연기', 6326),
             ('조차', 242),
             ('가볍다', 360),
             ('않다', 7718),
             ('너', 670),
             ('무재', 69),
             ('밓었', 1),
             ('다그', 75),
             ('래서', 20),
             ('보다', 40991),
             ('추천', 1180),
             ('한', 9615),
             ('다', 10077),
             ('교도소', 16),
             ('이야기', 2171),
             ('구먼', 11),
             ('솔직하다', 1199),
             ('재미', 3854),
             ('는', 16911),
             ('없다', 15519),
             ('평점', 6245),
             ('조정', 40),
             ('사이', 222),
             ('몬페', 2),
             ('그', 5667),
             ('의', 3073

In [17]:
total_cnt = len(tokenizer.word_index)
total_cnt

43770

In [35]:
#사용 단어 수를 지정하여 토큰화
tokenizer = Tokenizer(num_words = 30000)
tokenizer.fit_on_texts(train_df["token"])

In [36]:
threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq += value
    if value < threshold:
        rare_cnt += 1
        rare_freq += value

In [37]:
print("단어 집합의 크기 : ", total_cnt)
print("등장 빈도가 %d번 이하인 회귀 단어의 수 : %d" % (threshold-1, rare_cnt))
print("단어 집합에서 회귀 단어의 비율 : ", (rare_cnt / total_cnt) * 100)
print("전체 등장 빈도에서 회귀 단어 등장 빈도 비율 : ", (rare_freq / total_freq) * 100)

단어 집합의 크기 :  43770
등장 빈도가 2번 이하인 회귀 단어의 수 : 24340
단어 집합에서 회귀 단어의 비율 :  55.608864519076995
전체 등장 빈도에서 회귀 단어 등장 빈도 비율 :  1.5747898974035586


In [38]:
x_train = tokenizer.texts_to_sequences(train_df["token"])
x_test = tokenizer.texts_to_sequences(test_df["token"])

In [39]:
x_train

[[64, 473, 27, 278, 677],
 [965, 474, 54, 620, 2, 230, 1466, 37, 979, 694, 31],
 [403, 2460, 25033, 2330, 5684, 3, 237, 21, 19],
 [6507, 120, 8133, 234, 70, 11, 14, 39, 3619],
 [1041,
  19431,
  42,
  5,
  9158,
  37,
  7,
  849,
  2,
  2592,
  33,
  1126,
  256,
  4,
  14255,
  19432,
  7,
  1092,
  270,
  256],
 [739,
  5685,
  998,
  1405,
  443,
  159,
  1707,
  1640,
  11578,
  241,
  2,
  108,
  142,
  1109,
  9,
  58,
  261],
 [229, 5, 331, 8, 341, 503, 4],
 [142,
  1109,
  9,
  58,
  349,
  36,
  9801,
  14256,
  315,
  138,
  1600,
  1,
  385,
  135,
  242,
  20,
  817,
  29,
  597,
  592,
  536,
  493,
  3134,
  8134,
  24,
  1406,
  1406,
  1,
  50,
  11,
  301,
  9,
  14,
  37,
  48,
  4,
  52,
  24,
  721,
  1090,
  83],
 [110, 1, 14, 70, 16, 385, 113, 2],
 [1548, 39, 1, 217, 554, 97, 24, 21, 410, 1457, 378, 683, 18, 5686, 16],
 [399, 4704, 7, 183, 15, 27, 183, 15],
 [759,
  401,
  1243,
  220,
  389,
  5,
  2543,
  306,
  357,
  11579,
  11,
  621,
  2512,
  825,
  5,
  5

In [40]:
y_train = np.array(train_df["label"])
y_test = np.array(test_df["label"])

In [41]:
y_train

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [42]:
train_df["label"].values

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [43]:
length = np.array([len(x) for x in x_train])

In [44]:
print(np.mean(length), np.median(length), np.max(length), np.min(length))

13.13909230126688 10.0 75 0


## 모델

In [46]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, stratify = y_train,
                                                  test_size = 0.2, random_state = 42)

In [47]:
train_seq = pad_sequences(x_train, maxlen = 80)
val_seq = pad_sequences(x_val, maxlen = 80)
test_seq = pad_sequences(x_test, maxlen = 80)

In [48]:
model = keras.Sequential()
model.add(keras.layers.Embedding(30000, 100, input_shape = (80,)))
model.add(keras.layers.LSTM(128, dropout = 0.4, return_sequences = True))
model.add(keras.layers.LSTM(128, dropout = 0.4))
model.add(keras.layers.Dense(1, activation = "sigmoid"))

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:89: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [49]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 80, 100)             │       3,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 80, 128)             │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,248,961 (12.39 MB)

 Trainable params: 3,248,961 (12.39 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
adam = keras.optimizers.Adam(learning_rate = 0.005)
model.compile(optimizer = adam, loss = "binary_crossentropy", metrics = ["accuracy"])
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 3, restore_best_weights = True)

In [51]:
history = model.fit(train_seq, y_train, epochs = 100, batch_size = 128,
                   validation_data = (val_seq, y_val),
                   callbacks = [early_stopping_cb])

Epoch 1/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 177s 193ms/step - accuracy: 0.7878 - loss: 0.4394 - val_accuracy: 0.8482 - val_loss: 0.3398
Epoch 2/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 177s 197ms/step - accuracy: 0.8833 - loss: 0.2810 - val_accuracy: 0.8503 - val_loss: 0.3472
Epoch 3/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 213s 237ms/step - accuracy: 0.9056 - loss: 0.2291 - val_accuracy: 0.8494 - val_loss: 0.3832
Epoch 4/100
898/898 ━━━━━━━━━━━━━━━━━━━━ 215s 240ms/step - accuracy: 0.9216 - loss: 0.1957 - val_accuracy: 0.8473 - val_loss: 0.3892


In [57]:
def sent_predict(text):
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", text)
    text = okt.morphs(text, stem = True)
    encoded = tokenizer.texts_to_sequences([text])
    pad_new = pad_sequences(encoded, maxlen = 80)
    score = model.predict(pad_new)
    
    return score

In [58]:
sent_predict("잘도 그러겠다")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step


array([[0.44767293]], dtype=float32)

In [59]:
sent_predict("엿같고 참 좋아요")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


array([[0.58925676]], dtype=float32)

In [60]:
sent_predict("시간낭비하기 좋아요")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


array([[0.02318932]], dtype=float32)

In [61]:
sent_predict("돈낭비는 즐거워")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


array([[0.07015264]], dtype=float32)